# Similar Word Prediction
Word2Vec model development using HuggingFace dataset for predicting similar words, enhancing text analysis and NLP applications with accurate word embeddings.

## Installing Libraries

In [1]:
%%capture
!pip install datasets

## Importing Libraries

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset
from gensim.models import Word2Vec
from tqdm import tqdm

In [3]:
nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading Dataset

In [4]:
dataset_id = 'sentence-transformers/all-nli'
config = 'pair-score'

dataset = load_dataset(dataset_id, config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Preparing Dataset

In [5]:
dataset.shape

{'train': (942069, 3), 'dev': (19657, 3), 'test': (19656, 3)}

In [6]:
train_sentence1 = dataset['train']['sentence1']
train_sentence2 = dataset['train']['sentence2']
test_sentence1 = dataset['test']['sentence1']
test_sentence2 = dataset['test']['sentence2']
dev_sentence1 = dataset['dev']['sentence1']
dev_sentence2 = dataset['dev']['sentence2']

In [7]:
all_sentences = train_sentence1 + train_sentence2 + test_sentence1 + test_sentence2 + dev_sentence1 + dev_sentence2
all_sentences = list(set(all_sentences))

In [8]:
len(all_sentences)

1196336

## Preprocessing Data

In [9]:
# create linguistic_preprocessing function
def linguistic_preprocessing(text):

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    return lemmatized_tokens

In [10]:
sentences = []
for sent in tqdm(all_sentences):
  sentences.append(linguistic_preprocessing(sent))

sentences[:5]

100%|██████████| 1196336/1196336 [28:35<00:00, 697.26it/s]


[['dad', 'cherishes', 'new', 'baby', '.'],
 ['group',
  'people',
  'gather',
  'round',
  'coffe',
  'table',
  'costume',
  'party',
  '.'],
 ['man', 'walking', 'street', 'past', 'colorful', 'house', '.'],
 ['hud',
  'decided',
  'take',
  'step',
  'towards',
  'correcting',
  'final',
  'rule',
  '.'],
 ['everybody', 'knew', 'hijacking', '.']]

## Training Word2Vec Model

In [11]:
# Train a Word2Vec model with hyperparameters
model = Word2Vec(sentences, vector_size=300, window=5, min_count=10, sg=0, epochs=5)

## Predicting Similar Words

In [12]:
def get_similar_word(word):
  try:
    word = word.lower()
    similar_words = model.wv.most_similar(word, topn=5)
    for word, score in similar_words:
      print(f"{word.capitalize()}: {score:.2f}")
  except:
    print('No similar word found in vocabulary!')

In [13]:
get_similar_word('Cricket')

Rugby: 0.84
Badminton: 0.84
Lacrosse: 0.84
Ping-pong: 0.80
Dice: 0.79


In [14]:
get_similar_word('Book')

Novel: 0.65
Textbook: 0.63
Magazine: 0.61
Newspaper: 0.60
Bible: 0.58


In [15]:
get_similar_word('Dog')

Puppy: 0.83
Poodle: 0.81
Cat: 0.75
Chihuahua: 0.65
Canine: 0.65
